In [8]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [9]:
#Coloque o caminho em que está a pasta 'cg-proc' que foi clonada do repositório - NÃO EXCLUO OS OUTROS CAMINHOS, APENAS COMENTE.
# master_path = "C:/Users/bruno/"
master_path = "home/ken/KR/USP/7S/CG"

In [10]:
# Paths das imagens
one_each = "{}cg-proc/img/one_each.jpeg".format(master_path)
glued = "{}cg-proc/img/glued.jpeg".format(master_path)
mess = "{}cg-proc/img/mess.jpeg".format(master_path)
real_and_twentyFive = "{}cg-proc/img/real_and_twentyFive.jpeg".format(master_path)

In [11]:
# Dict para usar o nome da variável na hora de salvar o arquivo alterado.
images_dict = {
    'one_each': one_each,
    'glued': glued,
    'mess': mess,
    'real_and_twentyFive': real_and_twentyFive
}

In [12]:
#Funções de auxilio para plicar as transformações na imagem

def contrasteBrilho(image,alpha,beta):

  return cv2.convertScaleAbs(image, alpha, beta)


def rgb_to_gray(image):

    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return gray_image

def threshold(image, threshold_value):

    _, binarized_image = cv2.threshold(image, threshold_value, 255, cv2.THRESH_BINARY)
    return binarized_image

def adapt_threshold(image):

    binarized_image = cv2.adaptiveThreshold(image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 21, 4)
    return binarized_image

def negative(image):

    negative_image = cv2.bitwise_not(image)
    return negative_image

In [13]:
#Ajuste de brilho e contraste
for name, img_path in images_dict.items():
    image = cv2.imread(img_path)
    brilho = 100
    contraste = 1.5
    image = contrasteBrilho(image,brilho,contraste)

[ WARN:0@79.827] global loadsave.cpp:248 findDecoder imread_('home/ken/KR/USP/7S/CGcg-proc/img/one_each.jpeg'): can't open/read file: check file path/integrity
[ WARN:0@79.827] global loadsave.cpp:248 findDecoder imread_('home/ken/KR/USP/7S/CGcg-proc/img/glued.jpeg'): can't open/read file: check file path/integrity
[ WARN:0@79.885] global loadsave.cpp:248 findDecoder imread_('home/ken/KR/USP/7S/CGcg-proc/img/mess.jpeg'): can't open/read file: check file path/integrity
[ WARN:0@79.885] global loadsave.cpp:248 findDecoder imread_('home/ken/KR/USP/7S/CGcg-proc/img/real_and_twentyFive.jpeg'): can't open/read file: check file path/integrity


In [14]:
# Primeira etapa: carregando imagens e fazendo um primeiro threshold
for name, img_path in images_dict.items():
    image = cv2.imread(img_path)
    gray_image = rgb_to_gray(image)
    threshold_value = 135
    binarized_image = threshold(gray_image, threshold_value)
    cv2.imwrite("{}cg-proc/img/edited/t_hold_{}.jpeg".format(master_path, name), binarized_image)

[ WARN:0@80.242] global loadsave.cpp:248 findDecoder imread_('home/ken/KR/USP/7S/CGcg-proc/img/one_each.jpeg'): can't open/read file: check file path/integrity


error: OpenCV(4.9.0) /io/opencv/modules/imgproc/src/color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


In [ ]:
# Segunda etapa: um novo threshold para deixar a imagem mais "flat": sem as sombras das imagens
for name, img_path in images_dict.items():
    image = cv2.imread(img_path)
    gray_image = rgb_to_gray(image)
    binarized_image = adapt_threshold(gray_image)
    cv2.imwrite("{}cg-proc/img/edited/t_hold_adapt_{}.jpeg".format(master_path, name), binarized_image)

In [ ]:
# Terceira etapa: fazendo o negativo da imagem para trabalhar com as cores certas.
images_dict_edited = {
    'one_each_': "{}cg-proc/img/edited/t_hold_one_each.jpeg".format(master_path),
    'glued_': "{}cg-proc/img/edited/t_hold_glued.jpeg".format(master_path),
    'mess_': "{}cg-proc/img/edited/t_hold_mess.jpeg".format(master_path),
    'real_and_twentyFive_': "{}cg-proc/img/edited/t_hold_real_and_twentyFive.jpeg".format(master_path)
}

for name, img_path in images_dict_edited.items():
    image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    median_blurred = cv2.medianBlur(image,7)
    threshold_value = 135
    binarized_image = threshold(median_blurred, threshold_value)
    negative_image = negative(binarized_image)
    cv2.imwrite("{}cg-proc/img/edited/{}_negative.jpeg".format(master_path, name), negative_image)